In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

DEFAULT_RESOLUTION=30
DEFAULT_SIMPLE_ENERGY_ALPHA=2

In [ ]:
# Definition of a Curve
class Curve:
    # Constructor
    def __init__(self, list_of_points, closed=False):
        self.list_of_points = list_of_points
        self.closed = closed
        self.J = len(list_of_points)
    
    # Square Bracket Overload
    def __getitem__(self, key):
        return self.list_of_points[key % self.J]
    def __setitem__(self, key, value):
        self.list_of_points[key] = value
    
    # Length
    def __len__(self):
        return self.J
    
    # Curve Addition
    def __add__(self, otherCurve):
        if len(self) == len(otherCurve) and self.closed == otherCurve.closed:
            return Curve([self[i] + otherCurve[i] for i in range(len(self))])
    
    # Curve Length
    def curveLength(self):
        l = 0
        for i in range(self.J - 1):
            edgeLength = np.linalg.norm(self[i + 1] - self[i])
            l += edgeLength ** 2
        if self.closed:
            edgeLength = np.linalg.norm(self[-1] - self[0])
            l += edgeLength ** 2
        return l

In [ ]:
# Simple Curve Energy Version 2 (Averaged over 4 points)
def simpleCurveEnergy2(curve, alpha=DEFAULT_SIMPLE_ENERGY_ALPHA):
    J = curve.J

    integral = 0
    for i in range(J):
        for j in range(J):
            # Undefined for |i - j| <= 1
            if abs(i - j) <= 1 or abs(i - j + J) <= 1 or abs(i - j - J) <= 1:
                continue

            contribution = 1 / np.linalg.norm(curve[i] - curve[j]) ** alpha + 1 / np.linalg.norm(curve[i] - curve[j+1]) ** alpha
            contribution += 1 / np.linalg.norm(curve[i+1] - curve[j]) ** alpha + 1 / np.linalg.norm(curve[i+1] - curve[j+1]) ** alpha
            contribution /= 4
            integral += contribution * np.linalg.norm(curve[i] - curve[i+1]) * np.linalg.norm(curve[j] - curve[j+1])
    
    return integral


In [ ]:
# p, q, T are all array
def kernelalphabeta(p, q, T, alpha=2, beta=4):
    pmq = p - q
    numerator = np.linalg.norm(np.cross(T, pmq)) ** alpha
    denominator = np.linalg.norm(pmq) ** beta
    return numerator / denominator

# Pass a list of points.
def tangentPointEnergy(curve):
    J = len(curve)

    e = 0
    for i in range(J):
        for j in range(J):
            if abs(i - j) > 1 and abs(i - j + J) > 1 and abs(i - j - J) > 1:
                xi = curve[i % J]
                xipm = curve[(i+1) % J]
                xj = curve[j % J]
                xjpm = curve[(j+1) % J]
                xI = xipm - xi
                lI = np.linalg.norm(xI)
                TI = xI / lI
                kernelAB = 0
                kernelAB += kernelalphabeta(xi, xj, TI) 
                kernelAB += kernelalphabeta(xi, xjpm, TI) 
                kernelAB += kernelalphabeta(xipm, xj, TI) 
                kernelAB += kernelalphabeta(xipm, xjpm, TI)
                kernelAB *= 0.25 * lI * np.linalg.norm(xjpm-xj)
                e += kernelAB
                #print(f"{i}, {j}: kernelAB: {kernelAB}")

    return e

In [ ]:
def curveFromFourierCoefficients(xa, xb, ya, yb, za, zb, resolution=DEFAULT_RESOLUTION):
    J = len(xa) - 1

    def fourierFromCoefficient(an, bn):
        return lambda x : an[0]/2 + sum([np.cos(x)*an[i] + np.sin(x)*bn[i] for i in range(1, J + 1)])
    
    fx = fourierFromCoefficient(xa, xb)
    fy = fourierFromCoefficient(ya, yb)
    fz = fourierFromCoefficient(za, zb)

    points = []
    for i in range(resolution):
        theta = 2 * np.pi * i / resolution
        points.append(np.array([
            fx(theta),
            fy(theta),
            fz(theta)
        ]))
    
    return Curve(points, True)

In [ ]:
def curveFourierDifferential(xa, xb, ya, yb, za, zb, perturbation=0.001, resolution=DEFAULT_RESOLUTION, alpha=DEFAULT_SIMPLE_ENERGY_ALPHA, energyFunction=simpleCurveEnergy2):
    J = len(xa) - 1

    variables = np.concatenate([xa, xb, ya, yb, za, zb])
    differential = np.zeros(len(variables), dtype="float64")
    for i in range(len(variables)):
        temp = variables[i]
        variables[i] += perturbation
        xac, xbc, yac, ybc, zac, zbc = np.split(variables, 6)
        tempCurve = curveFromFourierCoefficients(xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
        energyP = energyFunction(tempCurve)
        variables[i] = temp
        variables[i] -= perturbation
        xac, xbc, yac, ybc, zac, zbc = np.split(variables, 6)
        tempCurve = curveFromFourierCoefficients(xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
        energyN = energyFunction(tempCurve)
        differential[i] = (energyP - energyN) / (2 * perturbation)
    
    return differential

In [ ]:
def curvePlot(curve):
    J = curve.J

    xpoints = []
    ypoints = []
    zpoints = []
    for i in range(J):
        xpoints.append(curve[i][0])
        ypoints.append(curve[i][1])
        zpoints.append(curve[i][2])
    
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    ax.plot(xpoints, ypoints, zpoints)
    ax.set_xlim((-2,2))
    ax.set_ylim((-2,2))
    ax.set_zlim((-2,2))
    plt.show()

In [ ]:
def fourierCurveSDM(xa, xb, ya, yb, za, zb, resolution=DEFAULT_RESOLUTION, alpha=DEFAULT_SIMPLE_ENERGY_ALPHA, M=100, stepsize=0.01, qPlot=False, energyFunction=simpleCurveEnergy2):
    x = np.concatenate([xa, xb, ya, yb, za, zb])
    for t in range(M):
        xac, xbc, yac, ybc, zac, zbc = np.split(x, 6)
        differential = curveFourierDifferential(xac, xbc, yac, ybc, zac, zbc, perturbation=0.0001, resolution=resolution, alpha=alpha, energyFunction=energyFunction)
        x = x - stepsize * differential

        if qPlot:
            tempcurve = curveFromFourierCoefficients(xac, xbc, yac, ybc, zac, zbc, resolution=resolution)
            print(energyFunction(tempcurve))
            curvePlot(tempcurve)
    

In [ ]:
xa = np.array([0, 1, 0, 0], dtype="float64")
xb = np.array([0, 0, 0, 0], dtype="float64")
ya = np.array([0, 0, 0, 0], dtype="float64")
yb = np.array([0, 1, 0, 0], dtype="float64")
za = np.array([0, 0, 0, 0], dtype="float64")
zb = np.array([0, 0, 0, 0], dtype="float64")

curve = curveFromFourierCoefficients(xa, xb, ya, yb, za, zb, resolution=10)
print(simpleCurveEnergy2(curve))
#curvePlot(curve)

In [ ]:
curveFourierDifferential(xa, xb, ya, yb, za, zb, 0.001, energyFunction=tangentPointEnergy)

In [ ]:
fourierCurveSDM(xa, xb, ya, yb, za, zb, resolution=10, alpha=1, M=1000, stepsize=0.04, qPlot=True, energyFunction=tangentPointEnergy)